# DESCARGA DE DATOS

In [2]:
#Create DatasetDict
from datasets import DatasetDict, Dataset, load_dataset
dataset_path = 'RikoteMaster/isear_augmented'
dataset_dict = load_dataset(dataset_path)

dataset_dict = dataset_dict.remove_columns('Augmented')
dataset_dict 


Found cached dataset parquet (/root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['Emotion', 'Text_processed'],
        num_rows: 752
    })
    test: Dataset({
        features: ['Emotion', 'Text_processed'],
        num_rows: 752
    })
    train: Dataset({
        features: ['Emotion', 'Text_processed'],
        num_rows: 10751
    })
})

In [3]:
#charge the train datasetDict as a df
df = dataset_dict['train'].to_pandas()
df.head()
#create id2label and label2id
id2label = {i: label for i, label in enumerate(df['Emotion'].unique())}
label2id = {label: i for i, label in enumerate(df['Emotion'].unique())}

#apply label2id to the datasetDict
dataset_dict = dataset_dict.map(lambda example: {'labels': label2id[example['Emotion']]}, remove_columns=['Emotion'])



Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-67b0a7624c7a5d2f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-161de27b49f6fdea.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-03e305454b1bb339.arrow


In [4]:
dataset_dict

DatasetDict({
    validation: Dataset({
        features: ['Text_processed', 'labels'],
        num_rows: 752
    })
    test: Dataset({
        features: ['Text_processed', 'labels'],
        num_rows: 752
    })
    train: Dataset({
        features: ['Text_processed', 'labels'],
        num_rows: 10751
    })
})

### Carga del tokenizador

In [5]:
from transformers import AutoTokenizer
model_ckpt = "bhadresh-savani/bert-base-go-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [6]:
def tokenize_text(examples):
    return tokenizer(examples["Text_processed"], padding="max_length")

In [7]:
dataset_dict = dataset_dict.map(tokenize_text, batched=True)
dataset_dict

Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-de0f2d38f00f62e0.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d12b0dfd191548b2.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c3255ff59bfc3ef6.arrow


DatasetDict({
    validation: Dataset({
        features: ['Text_processed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 752
    })
    test: Dataset({
        features: ['Text_processed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 752
    })
    train: Dataset({
        features: ['Text_processed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 10751
    })
})

In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    
    return {'eval_accuracy': acc, 'f1': f1}

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "weight_decay" : trial.suggest_float("weight_decay", 1e-6, 1e-1, log=True),

    }

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def model_init(trial):
    if 'model' in locals():
        del model
        torch.cuda.empty_cache()

    model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=len(id2label), ignore_mismatched_sizes=True).to(device)
    for param in model.base_model.parameters():
        param.requires_grad = False
    
    return model


def compute_objective(metrics):
    
    return metrics['eval_accuracy'] + metrics['f1']


batch_size = 16
epochs = 5

output_dir = './results_searching_hyperparameters'
logging_steps = len(dataset_dict['train']) // batch_size

args = TrainingArguments( 
                        output_dir=output_dir, 
                        num_train_epochs=epochs,
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        evaluation_strategy='epoch',
                        logging_steps=logging_steps,
                        fp16=True,
                        push_to_hub=False,
                        # prevent saving checkpoints
                        save_strategy="steps",
                        save_steps=int(1e9),  # a large number
                    )

trainer = Trainer(
    model=None,
    args=args,
    train_dataset= dataset_dict['train'],
    eval_dataset= dataset_dict['validation'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
)

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=20
)

# Save best_trial to a text file

best_trial_dict = {
    'run_id': best_trial.run_id,
    'objective': best_trial.objective,
    'hyperparameters': best_trial.hyperparameters
}

# Save best_trial_dict to a text file
import json
with open('best_trial_frozen.txt', 'w') as file:
    file.write(json.dumps(best_trial_dict, indent=4))  # indent=4 for pretty printing



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2023-07-14 08:14:04,550] A new study created in memory with name: no-name-f8cf4139-9976-42ab-be1b-b5d865ebc956
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shap

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.921000,1.829193,0.309840,0.284362
2,1.800000,1.739646,0.384309,0.361191
3,1.748200,1.698882,0.390957,0.372862
4,1.720900,1.677238,0.404255,0.385012
5,1.713200,1.671270,0.398936,0.383083


[I 2023-07-14 08:19:49,235] Trial 0 finished with value: 0.782019553079611 and parameters: {'learning_rate': 1.4488231669565197e-05, 'weight_decay': 0.002913414501542372}. Best is trial 0 with value: 0.782019553079611.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.983700,1.933830,0.198138,0.181690
2,1.907200,1.879359,0.256649,0.240071
3,1.871000,1.846732,0.293883,0.274941
4,1.849100,1.828356,0.327128,0.307326
5,1.841000,1.822428,0.337766,0.318337


[I 2023-07-14 08:25:33,009] Trial 1 finished with value: 0.6561028938473347 and parameters: {'learning_rate': 5.369775710518277e-06, 'weight_decay': 0.004674647270538714}. Best is trial 0 with value: 0.782019553079611.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_

...

[I 2023-07-14 09:26:07,166] Trial 17 pruned. 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.875400,1.754099,0.364362,0.344188


[I 2023-07-14 09:27:16,100] Trial 18 pruned. 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.785400,1.635501,0.401596,0.381708


[I 2023-07-14 09:28:25,001] Trial 19 pruned. 


In [8]:
!nvidia-smi

Thu Jul 13 11:29:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN V      Off  | 00000000:01:00.0 Off |                  N/A |
| 34%   45C    P8    26W / 250W |  10004MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=len(id2label), ignore_mismatched_sizes=True)
#freeze all the parameters except the last layer, be sure that you freeze all excepting the last layer
for param in model.base_model.parameters():
    param.requires_grad = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import EarlyStoppingCallback
batch_size = 16
epochs = 30

output_dir = './results_freezed'
logging_steps = len(dataset_dict['train']) // batch_size
args = TrainingArguments( output_dir=output_dir, 
                        num_train_epochs=epochs,
                        learning_rate=3.562121201880562e-04,
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        weight_decay=0.0007924379520012866,
                        evaluation_strategy='epoch',
                        save_strategy='epoch',
                        logging_steps=logging_steps,
                        fp16=True,
                        push_to_hub=False,
                        load_best_model_at_end=True,
                        metric_for_best_model='accuracy')

In [21]:
trainer = Trainer(model=model,
                  args=args,
                  train_dataset=dataset_dict['train'],
                  eval_dataset=dataset_dict['validation'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  callbacks = [EarlyStoppingCallback(early_stopping_patience=int(0.2*epochs))])

In [22]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.595100,1.531633,0.442819,0.430315
2,1.587500,1.514691,0.432181,0.420767
3,1.581100,1.524175,0.445479,0.441161
4,1.574300,1.530792,0.416223,0.416816
5,1.573800,1.515810,0.445479,0.433519
6,1.565500,1.509822,0.448138,0.438635
7,1.563100,1.502079,0.465426,0.452725
8,1.558000,1.511683,0.464096,0.454242
9,1.546200,1.491038,0.462766,0.455032
10,1.551900,1.495883,0.458777,0.445807


TrainOutput(global_step=12096, training_loss=1.5571194822510714, metrics={'train_runtime': 1307.1586, 'train_samples_per_second': 246.741, 'train_steps_per_second': 15.423, 'total_flos': 5.09190110148096e+16, 'train_loss': 1.5571194822510714, 'epoch': 18.0})

In [23]:
#push to hub
trainer.push_to_hub()

/home/mriciba/Projects/dipsy/BERTS/code/BERT/./results_freezed is already a clone of https://huggingface.co/RikoteMaster/results_freezed. Make sure you pull the latest changes with `repo.git_pull()`.


Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Jul13_12-41-57_2bfb4451df90/events.out.tfevents.1689252118.2bfb4451df90.283443.21:   0%|     …

Upload file training_args.bin:   0%|          | 1.00/3.87k [00:00<?, ?B/s]

To https://huggingface.co/RikoteMaster/results_freezed
   f0a7fe0..3799876  main -> main

To https://huggingface.co/RikoteMaster/results_freezed
   3799876..2563c18  main -> main



'https://huggingface.co/RikoteMaster/results_freezed/commit/37998761a70e46bb59eb1487e7155648e5274438'

In [ ]:
#predict on test set
preds = trainer.predict(dataset_dict['test'])
preds = preds.predictions.argmax(-1)
#calculate accuracy
acc = accuracy_score(test_df['Emotion'], preds)

print(acc)

In [ ]:
for param in model.bert.encoder.layer[-2:].parameters():
    param.requires_grad = False


In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()